In [ ]:
from emotion_recognition import EmotionRecognizer
from deep_emotion_recognition import DeepEmotionRecognizer

In [ ]:
import pyaudio
import os
import wave
from sys import byteorder
from array import array
from struct import pack
from sklearn.ensemble import GradientBoostingClassifier, BaggingClassifier

In [ ]:
from utils import get_best_estimators
import warnings
# import tensorflow as tf

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
THRESHOLD = 500
CHUNK_SIZE = 1024
FORMAT = pyaudio.paInt16
RATE = 16000

In [ ]:
SILENCE = 30

In [ ]:
def is_silent(snd_data):
    "Returns 'True' if below the 'silent' threshold"
    return max(snd_data) < THRESHOLD

In [ ]:
def normalize(snd_data):
    "Average the volume out"
    MAXIMUM = 16384
    times = float(MAXIMUM) / max(abs(i) for i in snd_data)
    r = array('h')
    for i in snd_data:
        r.append(int(i * times))
    return r

In [ ]:
def trim(snd_data):
    "Trim the blank spots at the start and end"
    def _trim(snd_data):
        snd_started = False
        r = array('h')
        for i in snd_data:
            if not snd_started and abs(i) > THRESHOLD:
                snd_started = True
                r.append(i)
            elif snd_started:
                r.append(i)
        return r

    # Trim to the left
    snd_data = _trim(snd_data)

    # Trim to the right
    snd_data.reverse()
    snd_data = _trim(snd_data)
    snd_data.reverse()
    return snd_data

In [ ]:
def add_silence(snd_data, seconds):
    "Add silence to the start and end of 'snd_data' of length 'seconds' (float)"
    r = array('h', [0 for i in range(int(seconds * RATE))])
    r.extend(snd_data)
    r.extend([0 for i in range(int(seconds * RATE))])
    return r

In [ ]:
def record():
    """
    Record a word or words from the microphone and 
    return the data as an array of signed shorts.
    Normalizes the audio, trims silence from the 
    start and end, and pads with 0.5 seconds of 
    blank sound to make sure VLC et al can play 
    it without getting chopped off.
    """
    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT, channels=1, rate=RATE,
                    input=True, output=True,
                    frames_per_buffer=CHUNK_SIZE)
    num_silent = 0
    snd_started = False
    r = array('h')
    while 1:
        # little endian, signed short
        snd_data = array('h', stream.read(CHUNK_SIZE))
        if byteorder == 'big':
            snd_data.byteswap()
        r.extend(snd_data)
        silent = is_silent(snd_data)
        if silent and snd_started:
            num_silent += 1
        elif not silent and not snd_started:
            snd_started = True
        if snd_started and num_silent > SILENCE:
            break
    sample_width = p.get_sample_size(FORMAT)
    stream.stop_stream()
    stream.close()
    p.terminate()
    r = normalize(r)
    r = trim(r)
    r = add_silence(r, 0.5)
    return sample_width, r

In [ ]:
def record_to_file(path):
    "Records from the microphone and outputs the resulting data to 'path'"
    sample_width, data = record()
    data = pack('<' + ('h' * len(data)), *data)
    wf = wave.open(path, 'wb')
    wf.setnchannels(1)
    wf.setsampwidth(sample_width)
    wf.setframerate(RATE)
    wf.writeframes(data)
    wf.close()

In [ ]:
def get_estimators_name(estimators):
    result = ['"{}"'.format(estimator.__class__.__name__) for estimator, _, _ in estimators]
    return ','.join(result), {estimator_name.strip('"'): estimator for estimator_name, (estimator, _, _) in
                              zip(result, estimators)}

In [ ]:
if __name__ == "__main__":
    estimators = get_best_estimators(True)
    estimators_str, estimator_dict = get_estimators_name(estimators)
    import argparse
    parser = argparse.ArgumentParser(description="""
                                    Testing emotion recognition system using your voice, 
                                    please consider changing the model and/or parameters as you wish.
                                    """)
    parser.add_argument("-e", "--emotions", help=
    """Emotions to recognize separated by a comma ',', available emotions are
                                            "neutral", "calm", "happy" "sad", "angry", "fear", "disgust", "ps" (pleasant surprise)
                                            and "boredom", default is "sad,neutral,happy"
                                            """, default="sad,neutral,happy,angry")
    parser.add_argument("-m", "--model", help=
    """
                                        The model to use, 8 models available are: {},
                                        default is "BaggingClassifier"
                                        """.format(estimators_str), default="BaggingClassifier")

    # Parse the arguments passed
    args = parser.parse_args()
    lst = ['sad','neutral','happy']
    # 'excited', 'assertive', 'encouraging', 'anxious', 'apologetic', 'concerned'
    features = ["mfcc", "chroma", "mel"]
    # detector = EmotionRecognizer(estimator_dict[args.model], emotions=lst, features=features, verbose=0)
    # detector.train()
    # print("Test accuracy score: {:.3f}%".format(detector.test_score() * 100))
    # print("Please talk")
    # deeprec = DeepEmotionRecognizer(emotions=['angry', 'sad', 'neutral', 'ps', 'happy'], n_rnn_layers=2,
    #                                 n_dense_layers=2, rnn_units=128, dense_units=128)
    deeprec = DeepEmotionRecognizer(emotions=['sad', 'neutral', 'happy'], n_rnn_layers=2,
                                    n_dense_layers=2, rnn_units=128, dense_units=128)
    # train the model
    deeprec.train()
    # # get the accuracy
    print("Test accuracy score: {:.3f}%".format(deeprec.test_score() * 100))
    print(deeprec.confusion_matrix(percentage=False, labeled=True))

    # filename = "data/my-custom/female1_1a_1_concerned.wav"
    # # filename2= "test.wav"
    # # record_to_file(filename)
    # result = detector.predict_proba(filename)
    # print("Results:")
    # for emotion, prob in result.items():
    #     print(f"{emotion}: {prob*100}%")
    # # print(result)